In [1]:
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [24]:
def readQuote( inpFileName ):
    return pd.read_csv( inpFileName, header=0, index_col=0 )

In [81]:
# Calculates running average over a number of days, including that day!
def calcRunAvg( myDf, col, inpNumDays ):
    
    # Make sure we are accesing valid average numbers
    numDays = int( inpNumDays )
    if ( numDays < 2 or 
         numDays > myDf.shape[0]-1 ):
        print 'Invalid number of days for running average: ', inpNumDays
        print 'Valid numbers: 2, ', myDF.shape[0]-1
        sys.exit()
        
    # Make sure we are accessing valid columns
    cols = myDf.columns.values
    if ( not col in cols ):
        print 'Invalid column for running average: ', col
        print 'Valid columns: ', cols
        sys.exit()
    
    # Store running average
    runAvg = np.zeros( [myDf.shape[0]] )
    
    # Loop until we hit limit for number of days
    for i in range( 0, myDf.shape[0]-numDays+1 ):
        # Running average 
        runAvg[i] = myDf.ix[ i:i+numDays, col ].mean()
        
    newCol       = 'avg_'+col+'_%i' % inpNumDays
    myDf[newCol] = runAvg

In [85]:
def highLow( myDf ):
    myDf['highlow'] = myQuote['high']-myQuote['low']
def openClose( myDf ):
    myDf['openclose'] = myQuote['close'] - myQuote['open']

In [2]:
inpFile = 'quotes/ibm.csv'

In [25]:
myQuote = readQuote( inpFile )

In [86]:
#foo=myQuote.columns.values

#print myQuote.ix[0:10]['close'].mean()
#print myQuote.ix[0:10,'close'].mean()

bar = myQuote.copy()
#print bar
bar.is_copy = False

highLow( bar )
openClose( bar )
calcRunAvg( bar, 'close', 3 )
print bar.head()

             close     volume    open    high       low  highlow  openclose  \
date                                                                          
2017/03/10  177.83  3100107.0  178.21  179.49  177.4200   2.0700      -0.38   
2017/03/09  177.18  5443665.0  179.15  179.25  175.8800   3.3700      -1.97   
2017/03/08  179.45  3557388.0  180.75  180.95  179.3000   1.6500      -1.30   
2017/03/07  180.38  2977496.0  180.71  181.29  180.1997   1.0903      -0.33   
2017/03/06  180.47  3199175.0  179.72  180.99  179.5700   1.4200       0.75   

            avg_close_3  
date                     
2017/03/10   178.153333  
2017/03/09   179.003333  
2017/03/08   180.100000  
2017/03/07   180.300000  
2017/03/06   180.350000  


In [87]:
# Covariance of all variables
# Break into chunks of ___ days, so average isn't skewed to general trend of rise in the market
# ^ Can use metrics to determine slumps, shooting up, constant growth, volatile
# Plot trends over time
# Test differrent time varying models